In [1]:
# define some input data locations

from os.path import join
path = '/home/nils/data/EUR-44'
files = ['tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19710101-19751231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19760101-19801231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19810101-19851231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19860101-19901231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19910101-19951231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19960101-20001231.nc',]
resource = [ join(path,nc) for nc in files ]

Execution with plain python (no birdhouse components included )

In [16]:
import icclim 

from ocgis import RequestDataset, OcgOperations, env
from ocgis.contrib.library_icclim import IcclimTG90p

env.OVERWRITE = True
########################################################################################################################
# Compute a custom percentile basis using ICCLIM. ######################################################################
########################################################################################################################

# Subset the input dataset to return the desired base period for the percentile basis.
variable = 'tas'
years = range(1971, 2001)
time_region = {'year': years}
rd = RequestDataset(uri=resource, variable=variable)
field = rd.get()
field.get_time_region(time_region)

# Calculate the percentile basis. The data values must be a three-dimensional array.
arr = field.variables[variable].value.squeeze()
dt_arr = field.temporal.value_datetime
percentile = 90
window_width = 5
percentile_dict = IcclimTG90p.get_percentile_dict(arr, dt_arr, percentile, window_width)

calc = [{'func': 'icclim_TG90p', 'name': 'TG90p', 'kwds': {'percentile_dict': percentile_dict}}]
calc_grouping = 'year'
ops = OcgOperations(dataset=rd, calc=calc, calc_grouping=calc_grouping, output_format='nc')
coll = ops.execute()
print coll

/home/nils/birdhouse/flyingpigeon/notebooks/ocgis_output/ocgis_output.nc


In [29]:
from flyingpigeon import indices as ind
#from flyingpigeon.indices import calc_indice_percentile

In [63]:
reload(ind)
nc_indice = ind.calc_indice_percentile(resources=resource, variable='tas', indices='TG90p',
                       period=(1971,2000), groupings='yr', dir_output=None)
print nc_indice


start ocgis module
start ocgis module
['/', 'h', 'o', 'm', 'e', '/', 'n', 'i', 'l', 's', '/', 'b', 'i', 'r', 'd', 'h', 'o', 'u', 's', 'e', '/', 'f', 'l', 'y', 'i', 'n', 'g', 'p', 'i', 'g', 'e', 'o', 'n', '/', 'n', 'o', 't', 'e', 'b', 'o', 'o', 'k', 's', '/', 'T', 'G', '9', '0', 'p', '_', 'E', 'U', 'R', '-', '4', '4', '_', 'I', 'C', 'H', 'E', 'C', '-', 'E', 'C', '-', 'E', 'A', 'R', 'T', 'H', '_', 'h', 'i', 's', 't', 'o', 'r', 'i', 'c', 'a', 'l', '_', 'r', '1', '2', 'i', '1', 'p', '1', '_', 'S', 'M', 'H', 'I', '-', 'R', 'C', 'A', '4', '_', 'v', '1', '_', 'd', 'a', 'y', '_', '1', '9', '7', '1', '0', '1', '0', '1', '-', '2', '0', '0', '0', '1', '2', '3', '1', '.', 'n', 'c']


In [59]:
call(resource=nc_indice, prefix='nc_reference', time_region=time_region, output_format='nc')

start ocgis module


'/home/nils/birdhouse/flyingpigeon/notebooks/nc_reference.nc'

In [ ]:
from ocgis.contrib.library_icclim import IcclimTG90p
from flyingpigeon.ocgis_module import call
from flyingpigeon import utils as ut
from numpy import ma 
    
years = range(1971, 2001)
time_region = {'year': years} 

nc_reference = call(resource=resource, prefix='nc_reference',time_region=time_region, output_format='nc')

In [50]:
arr = ut.get_values(nc_files=nc_reference)
dt_arr = ut.get_time(nc_file=nc_reference)

In [51]:
arr = ma.masked_array(arr)
dt_arr = ma.masked_array(dt_arr)

percentile = 90
window_width = 5
percentile_dict = IcclimTG90p.get_percentile_dict(arr, dt_arr, percentile, window_width)

In [52]:
calc = [{'func': 'icclim_TG90p', 'name': 'TG90p', 'kwds': {'percentile_dict': percentile_dict}}]
calc_grouping = 'year'
coll = call(resource=resource, prefix='flyingpigeon_modules', calc=calc, calc_grouping=calc_grouping, output_format='nc')
print coll

start ocgis module
/home/nils/birdhouse/flyingpigeon/notebooks/flyingpigeon_modules.nc


In [49]:
nc_reference

'/home/nils/birdhouse/flyingpigeon/notebooks/nc_reference.nc'

Execute WPS icclim processes with PHYTHON 

In [1]:
from owslib.wps import WebProcessingService, monitorExecution
wps = WebProcessingService(url="http://localhost:8093/wps", verbose=False, skip_caps=False)

Explore some informations:

In [2]:
wps.identification.title

'Flyingpigeon'

In [3]:
for process in wps.processes:
    process.identifier, process.title

In [4]:
for process in wps.processes:
    print '%s \t : %s '% (process.identifier, process.abstract) 

visualisation 	 : Just testing a nice script to visualise some variables 
sdm 	 : Species distribution model (SDM) 
weatherregimes 	 : Weather Regimes based on pressure patterns (kmean method) 
extractpoints 	 : Extract Timeseries for specified coordinates from grid data 
segetalflora 	 : Species biodiversity of segetal flora. Imput files: variable:tas , domain: EUR-11 or EUR-44 
indices_single 	 : This process calculates climate indices based on one single variable. 
indices_percentile 	 : This process calculates climate indices based on one single variable and based on percentils of a referece period. 
subset_countries 	 : This process returns only the given polygon from input netCDF files. 
eobs_to_cordex 	 : downloads EOBS data in adaped CORDEX format 
ensembleRobustness 	 : Calculates the robustness as the ratio of noise to signal in an ensemle of timeseries 
analogs 	 : Search for days with analog pressure pattern 
fetch 	 : This process downloads resources (limited to 50GB)     

In [12]:
from flyingpigeon import indices #import calc_indice_percentile as cip
reload(indices)

<module 'flyingpigeon.indices' from '/home/nils/birdhouse/flyingpigeon/flyingpigeon/indices.pyc'>

In [ ]:
from flyingpigeon.utils import get_values, get_time

arr = get_values(nc_ref_vals)
dt_arr = get_time(nc_ref_vals)
percentile = 90
window_width = 5

arr = np.ma.masked_array(arr)
dt_arr = np.ma.masked_array(dt_arr)


In [55]:
percentile_dict = IcclimTG90p.get_percentile_dict(arr, dt_arr, percentile, window_width)

In [63]:
nc_indices =  call(resource=resource, variable='tas',  prefix='indices', calc=calc, calc_grouping=calc_grouping, output_format='nc')

start ocgis module


In [ ]:
import numpy as np

In [13]:
#np.ma.masked_array?


#reload(indices)

#indices.calc_indice_percentile(resource=resource, variable=None, prefix=None, indices='TG90p', period=None, groupings='yr')

In [ ]:
execute = wps.execute(
    identifier="indices_percentile",
    inputs=[
    ('indices','TG90p'),
    ('groupings','yr'),
    ('resource',resource[0]),
    ('resource',resource[1]),
    ('resource',resource[2]),
    ('resource',resource[3]),
    ('resource',resource[4]),
    ('resource',resource[5]),
    ])

# check process if completed ...
monitorExecution(execute, sleepSecs=5)

o = execute.processOutputs[0]
print o.reference